In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
semiotic = slg.Cenematic("en_bnc",requested_cpu=4)

Vocabulary will not be loaded from file.



In [3]:
semiotic.vocab.build(vocab_size = -50, parallel=True, corpus_length=80000, save=True, save_step=10, truncate_best_lens=500)

Computing in parallel
Normalize and jobs data...
... computed in 4.011059999465942 secs.

Build alphabet...
... computed in 0.0005099773406982422 secs.

Alphabet Size: 103
Special Tokens Size: 5
Terms to compute: 50

Enter loop


Saving intermediate results...
... computed in 0.7170660495758057 secs.
Intermediate vocabulary saved to models/en_bnc/vocabulary/110

Saving intermediate results...
... computed in 0.9167921543121338 secs.
Intermediate vocabulary saved to models/en_bnc/vocabulary/120

Saving intermediate results...
... computed in 1.430049180984497 secs.
Intermediate vocabulary saved to models/en_bnc/vocabulary/130

Saving intermediate results...
... computed in 1.067716121673584 secs.
Intermediate vocabulary saved to models/en_bnc/vocabulary/140

Saving intermediate results...
... computed in 1.0785210132598877 secs.
Intermediate vocabulary saved to models/en_bnc/vocabulary/150


Compute freq...
... computed in 1.1700410842895508 secs.

Vocabulary built
Vocabulary saved to models/en_bnc/vocabulary


In [3]:
%load_ext line_profiler

In [4]:
%lprun -f semiotic.vocab.build semiotic.vocab.build(vocab_size = 500, parallel=True, corpus_length=80000, save=False, save_step=10, truncate_best_lens=500)

Computing in parallel
Normalize and jobs data...
... computed in 4.66138482093811 secs.

Build alphabet...
... computed in 0.02866983413696289 secs.

Alphabet Size: 103
Special Tokens Size: 5
Terms to compute: 392

Enter loop



Compute freq...
... computed in 1.5186328887939453 secs.

Vocabulary built


Timer unit: 1e-06 s

Total time: 93.7908 s
File: /Users/Gianni/semiolog/semiolog/vocabulary.py
Function: build at line 1108

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1108                                               def build(
  1109                                                   self,
  1110                                                   corpus = None,
  1111                                                   vocab_size = None,
  1112                                                   special_tokens = None,
  1113                                                   save = False,
  1114                                                   save_step = None,
  1115                                                   truncate_best_lens = None,
  1116                                                   progress_bar = True,
  1117                                                   resume_merges = False,
  1118                                                   parallel = T